In [1]:
import pandas as pd
from cvxpy import *
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from tiingo import TiingoClient
pd.core.common.is_list_like = pd.api.types.is_list_like
config = {}
config['session'] = True
config['api_key'] = "9606b7b7c092fa68b5726d3feee8d44f40354fe3"
client = TiingoClient(config)

In [2]:
start_date='2005-01-01'
end_date='2019-01-31'
tickers = ['SPY','EFA','EEM','IEF','TLT','GLD','DBC','VNQ','HYG','LQD']

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
prices = pd.DataFrame()

for ticker in tickers:
#     prices[ticker] = get_historical_data(ticker, start=start_date, end=end_date, output_format='pandas').loc[:,'close']
    prices[ticker] = client.get_dataframe(ticker, startDate=start_date, endDate=end_date).loc[:,'adjClose']
print(prices.head())

                                 SPY        EFA        EEM        IEF  \
date                                                                    
2005-01-03 00:00:00+00:00  90.341626  35.796538  17.139783  56.772686   
2005-01-04 00:00:00+00:00  89.237701  35.110306  16.612075  56.419190   
2005-01-05 00:00:00+00:00  88.621906  35.087807  16.408715  56.512567   
2005-01-06 00:00:00+00:00  89.072488  35.087807  16.397560  56.565924   
2005-01-07 00:00:00+00:00  88.944823  34.930311  16.429308  56.519236   

                                 TLT    GLD  DBC        VNQ  HYG        LQD  
date                                                                         
2005-01-03 00:00:00+00:00  54.138698  43.02  NaN  29.437298  NaN  60.774245  
2005-01-04 00:00:00+00:00  53.571321  42.74  NaN  28.994870  NaN  60.433151  
2005-01-05 00:00:00+00:00  53.858060  42.67  NaN  28.031007  NaN  60.481879  
2005-01-06 00:00:00+00:00  53.894665  42.15  NaN  28.246955  NaN  60.525192  
2005-01-07 00:00:00+

In [8]:
prices = prices.dropna() # Shortcut to grab the most common starting date
returns = prices.pct_change() # convert prices to daily returns
returns = returns.iloc[1:,] # dropping the first day which is NA not using dropna() because want to catch that if it occurs latter in the dataset 
returns_ann = ((1 + returns.mean())**252) -1  # annualize returns

In [15]:
mu = np.array(returns_ann).reshape(-1,1)
Sigma = np.array(returns.cov() * 252)
n = len(tickers)

In [16]:
assert mu.shape == (n, 1)
assert Sigma.shape == (n,n)

In [24]:
from cvxpy import *
w = Variable(n)
# gamma = Parameter(nonneg=True)
gamma = 1
ret = mu.T*w
risk = quad_form(w, Sigma)
prob = Problem(Maximize(ret - gamma*risk), 
               [sum(w) == 1,
                w >= 0.05,
               w <= 0.4])

In [25]:
prob.solve()
print(pd.DataFrame(w.value, index=tickers, columns=['weights']))

     weights
SPY  0.25169
EFA  0.05000
EEM  0.05000
IEF  0.05000
TLT  0.34831
GLD  0.05000
DBC  0.05000
VNQ  0.05000
HYG  0.05000
LQD  0.05000


In [ ]:
# Compute trade-off curve.
SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)
for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    risk_data[i] = sqrt(risk).value
    ret_data[i] = ret.value

In [ ]:
# Plot long only trade-off curve.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

markers_on = [29, 40]
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(risk_data, ret_data, 'g-')
for marker in markers_on:
    plt.plot(risk_data[marker], ret_data[marker], 'bs')
    ax.annotate(r"$\gamma = %.2f$" % gamma_vals[marker], xy=(risk_data[marker]+.08, ret_data[marker]-.03))
for i in range(n):
    plt.plot(sqrt(Sigma[i,i]).value, mu[i], 'ro')
plt.xlabel('Standard deviation')
plt.ylabel('Return')
plt.show()